<a href="https://colab.research.google.com/github/weso500/IOT-Anomaly-Detection/blob/main/Single_Image_VAE_DV_DT_GenLatent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Obtained intially from https://keras.io/examples/generative/vae/ and https://colab.research.google.com/github/keras-team/keras-io/blob/master/examples/generative/ipynb/vae.ipynb

In [1]:
pip install --upgrade keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.6 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1


In [2]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import numpy as np
import tensorflow as tf
import keras
from keras import ops
from keras import layers

In [3]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.seed_generator = keras.random.SeedGenerator(1337)

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = ops.shape(z_mean)[0]
        dim = ops.shape(z_mean)[1]
        epsilon = keras.random.normal(shape=(batch, dim), seed=self.seed_generator)
        return z_mean + ops.exp(0.5 * z_log_var) * epsilon

Use this notebook as example for obtaining images: https://colab.research.google.com/drive/1EOFDisibXb9a4A3lZ_2z4Jbhlnx0OVy6 The below takes care of obtaining images

In [4]:
latent_dim = 3

encoder_inputs = keras.Input(shape=(256, 256, 3))
x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 256, 256, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 128, 128, 32)   │            896 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 64, 64, 64)     │         18,496 │ conv2d[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten (Flatten)         │ (None, 262144)         │              0 │ conv2d_1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 128)            │     33,554,560 │ flatten[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ z_mean (Dense)            │ (None, 3)              │            387 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ z_log_var (Dense)         │ (None, 3)              │            387 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sampling (Sampling)       │ (None, 3)              │              0 │ z_mean[0][0],          │
│                           │                        │                │ z_log_var[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 33,574,726 (128.08 MB)

 Trainable params: 33,574,726 (128.08 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(64 * 64 * 64, activation="relu")(latent_inputs)
x = layers.Reshape((64, 64, 64))(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv2DTranspose(3, 3, activation="sigmoid", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 3)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 262144)              │       1,048,576 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 64, 64, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose (Conv2DTranspose)   │ (None, 128, 128, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_1 (Conv2DTranspose) │ (None, 256, 256, 32)        │          18,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_2 (Conv2DTranspose) │ (None, 256, 256, 3)         │             867 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,104,835 (4.21 MB)

 Trainable params: 1,104,835 (4.21 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
from tensorflow.keras.layers import Layer
class KLLossLayer(Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
        kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))

        return kl_loss

class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
        self.kl_loss_layer = KLLossLayer()

    def call(self, inputs):  # Add the call method
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        return reconstructed

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = ops.mean(
                ops.sum(
                    keras.losses.binary_crossentropy(data, reconstruction),
                    axis=(1, 2),
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - ops.square(z_mean) - ops.exp(z_log_var))
            kl_loss = ops.mean(ops.sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        kl_loss = self.kl_loss_layer([z_mean, z_log_var])
        return reconstructed, kl_loss, z_mean, z_log_var

    def predict(self, x):
        reconstructed, kl_loss, z_mean, z_log_var = super().predict(x)
        return reconstructed, kl_loss, z_mean, z_log_var


In [7]:
!unzip /content/Gold_Folder_Final.zip

Streaming output truncated to the last 5000 lines.
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/108Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/109Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/10Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/110Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/111Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/112Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/113Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/114Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/115Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGB

In [8]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/Gold_Folder_Final/Training_Data/Normal/Spectrograms_Normal_DV_DT',
  image_size=(256, 256),
  batch_size=32,
  label_mode=None)


normalization_layer = layers.Rescaling(scale= 1./255)

normalized_ds = train_ds.map(lambda x: normalization_layer(x))

Found 500 files.


In [9]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_DV_DT',
  image_size=(256, 256),
  shuffle=False,
  batch_size=1,
  label_mode=None)

file_paths = test_ds.file_paths
print(file_paths)

normalization_layer = layers.Rescaling(scale= 1./255)

normalized_test_ds = test_ds.map(lambda x: normalization_layer(x))

Found 190 files.
['/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_DV_DT/500Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_DV_DT/501Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_DV_DT/502Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_DV_DT/503Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_DV_DT/504Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_DV_DT/505Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_DV_DT/506Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_DV_DT/507Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_DV_DT/508Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_DV_DT/509Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_DV_DT/510Normal.png', '/content/Gold_Folder_Fin

In [10]:
test_anomaly_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/Gold_Folder_Final/Test_Data/Anomaly/Spectrograms_Anomaly_DV_DT',
  image_size=(256, 256),
  shuffle=False,
  batch_size=1,
  label_mode=None)

file_paths = test_ds.file_paths
print(file_paths)

normalization_layer = layers.Rescaling(scale= 1./255)

normalized_anomaly_test_ds = test_anomaly_ds.map(lambda x: normalization_layer(x))

Found 182 files.
['/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_DV_DT/500Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_DV_DT/501Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_DV_DT/502Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_DV_DT/503Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_DV_DT/504Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_DV_DT/505Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_DV_DT/506Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_DV_DT/507Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_DV_DT/508Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_DV_DT/509Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_DV_DT/510Normal.png', '/content/Gold_Folder_Fin

In [11]:
#(x_train, _), (x_test, _) = keras.datasets.mnist.load_data()
#mnist_digits = np.concatenate([x_train, x_test], axis=0)
#mnist_digits = np.expand_dims(mnist_digits, -1).astype("float32") / 255

vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(normalized_ds, epochs=15, batch_size=32)

Epoch 1/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 31s 732ms/step - kl_loss: 262152192000.0000 - loss: 262153224192.0000 - reconstruction_loss: 1080612.0000
Epoch 2/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 15s 86ms/step - kl_loss: 12.1688 - loss: 45075.3086 - reconstruction_loss: 45063.1406
Epoch 3/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - kl_loss: 2.3469 - loss: 44708.6953 - reconstruction_loss: 44706.3477
Epoch 4/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - kl_loss: 0.8935 - loss: 44358.3164 - reconstruction_loss: 44357.4258
Epoch 5/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 85ms/step - kl_loss: 1.4620 - loss: 44025.6445 - reconstruction_loss: 44024.1875
Epoch 6/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 103ms/step - kl_loss: 1.8674 - loss: 43708.9922 - reconstruction_loss: 43707.1250
Epoch 7/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - kl_loss: 4.3100 - loss: 43407.1289 - reconstruction_loss: 43402.8164
Epoch 8/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 87ms/step - kl_loss: 40.1170 - loss: 42974.9531 - reconstruction_loss: 42934.832

In [12]:
train_ds_test_ = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/Gold_Folder_Final/Training_Data/Normal/Spectrograms_Normal_DV_DT',
  image_size=(256, 256),
  batch_size=1,
  label_mode=None)


normalization_layer = layers.Rescaling(scale= 1./255)

normalized_ds_test1 = train_ds_test_.map(lambda x: normalization_layer(x))

Found 500 files.


In [13]:
pip install sewar

  Preparing metadata (setup.py) ... done
  Created wheel for sewar: filename=sewar-0.4.6-py3-none-any.whl size=11419 sha256=9cfcaaadd6e1922d69388571bc2da2d8d9b57c1d9ea43b7565ac3777fa2db1f1
  Stored in directory: /root/.cache/pip/wheels/3f/af/02/9c6556ba287b62a945d737def09b8b8c35c9e1d82b9dfae84c
Successfully built sewar


In [14]:
import cv2
from urllib.request import AbstractBasicAuthHandler
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sewar.full_ref import mse, rmse, psnr, uqi, ssim, ergas, scc, rase, sam, msssim, vifp

#mse_values_normal = [] # Create an empty list to store mse values
#mse_values_anomaly = []
mse_values = []
kldiv1 = []
zmean1 = []
zmean2 = []
zmean3 = []
zlogvar1 = []
zlogvar2 = []
zlogvar3 = []

for elem in normalized_ds_test1.take(500):
  for i in range(len(elem)):
    elem_np = elem.numpy()
    exp = vae.predict(elem)
    exp_np = tf.get_static_value(exp)
    #plt.imshow(np.squeeze(elem_np))
    #plt.show()
    #plt.imshow(np.squeeze(exp_np[0]))
    #plt.show()
    #x = cv2.PSNR(elem_np, exp_np[0])
    x = mse(elem_np, exp_np[0])
    print(x)
    print(exp_np[1])
    kldiv1.append(exp_np[1])
    print(exp_np[2])
    zmean1.append(exp_np[2][0][0])
    zmean2.append(exp_np[2][0][1])
    zmean3.append(exp_np[2][0][2])
    print(exp_np[3])
    zlogvar1.append(exp_np[3][0][0])
    zlogvar2.append(exp_np[3][0][1])
    zlogvar3.append(exp_np[3][0][2])
    mse_values.append(x) # Append each mse value to the list



1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
0.02946900853236122
445.8329
[[ -0.2517875  -29.810862     0.09519776]]
[[-0.43538827 -3.5608158  -0.7722685 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
0.03222150846946624
415.05112
[[ -0.2651864  -28.761827     0.09587773]]
[[-0.4587385 -3.4799476 -0.663253 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
0.02876698278336956
448.01593
[[ -0.23392926 -29.884808     0.08885392]]
[[-0.4254127 -3.5398502 -0.7450715]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
0.03080479363209712
438.5951
[[ -0.25060853 -29.568075     0.0721999 ]]
[[-0.38384324 -3.5110216  -0.7923643 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
0.03007138766624426
450.85544
[[ -0.2700387 -29.978363    0.0348893]]
[[-0.404105  -3.597125  -0.7797814]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
0.029829810009215894
433.8715
[[ -0.25098976 -29.408934     0.10432824]]
[[-0.40630347 -3.4741457  -0.7188781 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
0.0315328311986939
428.35965
[[ -0.26980767 -29.220371     0.04541119]]


In [15]:
import pandas as pd

dict = {'MSE': mse_values, 'KL Loss': kldiv1, 'Zmean1': zmean1, 'Zmean2': zmean2, 'Zmean3': zmean3, 'Zlogvar1': zlogvar1,'Zlogvar2': zlogvar2, 'Zlogvar3': zlogvar3 }

df = pd.DataFrame(dict)

# saving the dataframe
df.to_csv('DV_DT_train.csv')

In [16]:
import cv2
from urllib.request import AbstractBasicAuthHandler
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sewar.full_ref import mse, rmse, psnr, uqi, ssim, ergas, scc, rase, sam, msssim, vifp

#mse_values_normal = [] # Create an empty list to store mse values
#mse_values_anomaly = []
mse_values = []
kldiv1 = []
zmean1 = []
zmean2 = []
zmean3 = []
zlogvar1 = []
zlogvar2 = []
zlogvar3 = []

for elem in normalized_test_ds.take(190):
  for i in range(len(elem)):
    elem_np = elem.numpy()
    exp = vae.predict(elem)
    exp_np = tf.get_static_value(exp)
    #plt.imshow(np.squeeze(elem_np))
    #plt.show()
    #plt.imshow(np.squeeze(exp_np[0]))
    #plt.show()
    #x = cv2.PSNR(elem_np, exp_np[0])
    x = mse(elem_np, exp_np[0])
    print(x)
    print(exp_np[1])
    kldiv1.append(exp_np[1])
    print(exp_np[2])
    zmean1.append(exp_np[2][0][0])
    zmean2.append(exp_np[2][0][1])
    zmean3.append(exp_np[2][0][2])
    print(exp_np[3])
    zlogvar1.append(exp_np[3][0][0])
    zlogvar2.append(exp_np[3][0][1])
    zlogvar3.append(exp_np[3][0][2])
    mse_values.append(x) # Append each mse value to the list

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
0.02532167363929037
574.7451
[[ -0.3750976  -33.851772    -0.16927376]]
[[-0.08877652 -3.9225087  -1.0991551 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
0.026655012732505996
573.0211
[[ -0.40606984 -33.799725    -0.18480149]]
[[-0.09238165 -3.9660823  -1.0996512 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
0.03465971790386468
408.79675
[[-2.8813490e-01 -2.8543957e+01  1.5664659e-02]]
[[-0.41887516 -3.4706323  -0.65504605]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
0.035535610981804085
412.71085
[[-2.9504642e-01 -2.8679829e+01  2.8669916e-02]]
[[-0.4339463 -3.5037112 -0.677738 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
0.031318875059337385
438.3546
[[-2.7908000e-01 -2.9559687e+01  2.8416239e-02]]
[[-0.47352463 -3.5638235  -0.6392718 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
0.030156496157373775
444.91666
[[ -0.2795752  -29.780489     0.03018864]]
[[-0.48782223 -3.5930052  -0.6123782 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
0.030636696213313636
445.43536
[[-2.9609

In [17]:
import pandas as pd

dict = {'MSE': mse_values, 'KL Loss': kldiv1, 'Zmean1': zmean1, 'Zmean2': zmean2, 'Zmean3': zmean3, 'Zlogvar1': zlogvar1,'Zlogvar2': zlogvar2, 'Zlogvar3': zlogvar3 }

df2 = pd.DataFrame(dict)

# saving the dataframe
df2.to_csv('DV_DT_testnorm.csv')

In [18]:
import cv2
from urllib.request import AbstractBasicAuthHandler
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sewar.full_ref import mse, rmse, psnr, uqi, ssim, ergas, scc, rase, sam, msssim, vifp

#mse_values_normal = [] # Create an empty list to store mse values
#mse_values_anomaly = []
mse_values = []
kldiv1 = []
zmean1 = []
zmean2 = []
zmean3 = []
zlogvar1 = []
zlogvar2 = []
zlogvar3 = []

for elem in normalized_anomaly_test_ds.take(190):
  for i in range(len(elem)):
    elem_np = elem.numpy()
    exp = vae.predict(elem)
    exp_np = tf.get_static_value(exp)
    #plt.imshow(np.squeeze(elem_np))
    #plt.show()
    #plt.imshow(np.squeeze(exp_np[0]))
    #plt.show()
    #x = cv2.PSNR(elem_np, exp_np[0])
    x = mse(elem_np, exp_np[0])
    print(x)
    print(exp_np[1])
    kldiv1.append(exp_np[1])
    print(exp_np[2])
    zmean1.append(exp_np[2][0][0])
    zmean2.append(exp_np[2][0][1])
    zmean3.append(exp_np[2][0][2])
    print(exp_np[3])
    zlogvar1.append(exp_np[3][0][0])
    zlogvar2.append(exp_np[3][0][1])
    zlogvar3.append(exp_np[3][0][2])
    mse_values.append(x) # Append each mse value to the list

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
0.03737092606684856
358.66733
[[ -0.47701463 -26.700459     0.7419782 ]]
[[-1.3316141 -3.853385   0.5342286]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
0.04437355003397463
738.79395
[[ -0.71437395 -38.34608      0.04326876]]
[[-1.5390028 -4.763166   1.5410732]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
0.03674678103955219
572.4098
[[ -0.4441882  -33.755207     0.61439854]]
[[-1.636265  -4.2349396  1.0183259]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
0.02454352407241663
568.5691
[[ -0.36526856 -33.66818     -0.20146552]]
[[ 0.01035101 -3.9096293  -1.181353  ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
0.05099856488288704
740.5611
[[ -0.56771195 -38.384815     0.26166448]]
[[-1.7116281 -4.607265   1.7119819]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
0.035130067279110305
436.34073
[[ -0.43609795 -29.456446     0.81566197]]
[[-1.539832  -4.0006857  0.753166 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
0.029648854346481013
435.47235
[[ -0.25489768 -29.462118     0.12851438]]

In [19]:
import pandas as pd

dict = {'MSE': mse_values, 'KL Loss': kldiv1, 'Zmean1': zmean1, 'Zmean2': zmean2, 'Zmean3': zmean3, 'Zlogvar1': zlogvar1,'Zlogvar2': zlogvar2, 'Zlogvar3': zlogvar3 }

df3 = pd.DataFrame(dict)

# saving the dataframe
df3.to_csv('DV_DT_testanom.csv')

In [ ]:
from google.colab import files
files.download('/content/B+FLUX_testanom.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('/content/B+FLUX_testnorm.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('/content/B+_FLUX_train.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>